# Case Studies: Creating LAMMPS Simulations of Hydrocarbon polymers

In this example, we use NNMDKit to create LAMMPS data and input files for molecular dynamics (MD) simulations of hydrocarbon polymers. We focus only on hydrocarbon polymers at this stage because our neural network force field was trained against mostly hydrocarbon polymeric systems  (thanks to Christopher Kuenneth for developing the force field!)


## Step 1: Importing NNMDKit and defining SMILES strings

In [ ]:
import nnmdkit

smiles = [
    '*CC*', '*CC(*)C', '*CC(*)CC', '*CC(*)CCC', '*CC(*)CCCC', '*CC(*)c1ccccc1'
]

First, NNMDKit should be able to imported if it's installed correctly. Secondly, we put SMILES strings of polymers of interest into a list, which we can later loop over to create input files and folders for each polymeric system.

## Step 2: Creating `System` objects and corresponding LAMMPS data files

In [ ]:
for s in smiles:
    sys = nnmdkit.System(smiles=s, mw=1000, ntotal=3000, density=0.85)
    data = sys.write_data(output_dir=s)


For each system/SMILES string, we initialize a `System` object, and for each `System`, four attributes have to be defined, including:
- `smiles` - polymer SMILES string
- `mw` - Polymer molecular weight
- `ntotal` - Total number of atoms
- `density` - Initial density of the system ($g/cm^3$)

Here, SMILES strings are provided through looping over the list defined in the first step; molecular weight and total number of atoms are chosen as 1000 and 3000, respectively; initial density is set to $0.85g/cm^3$ which is the density of polyethlyene observed from previous MD simulations.

After `System`'s are defined, we call the `write_data` function with the output directory (`output_dir`) specified to create the LAMMPS data file for each system. Here, we name each folder with the system's SMILES string. We will later put all necessary files for a LAMMPS run to the same folder. Note that the `write_data` function returns the data file name, which is fetched by `data` in this example. This `data` string will later be used to initialize `LAMMPS` objects.

## Step 3: Creating `Lammps` objects and corresponding LAMMPS input files

In [ ]:
Tmax = 800
Pmax = 100
Tfinal = 300
Pfinal = 1

eq_step = [
    ['nvt', 50000, Tmax],
    ['npt', 50000, Tfinal, Pmax],
    ['npt', 50000, Tfinal, Pfinal],
    ['nvt', 50000, Tmax],
    ['npt', 50000, Tfinal, Pmax],
    ['npt', 50000, Tfinal, Pfinal],
    ['nvt', 50000, Tmax],
    ['npt', 50000, Tfinal, Pmax],
    ['npt', 50000, Tfinal, Pfinal]
]

for s in smiles:
    lmp = nnmdkit.Lammps(data, NN_POTENTIAL='potential_saved')
    lmp.add_procedure('minimization', min_style='cg')
    lmp.add_procedure('equilibration', eq_step=eq_step)
    lmp.add_procedure('Tg_measurement', Tinit=400, Tfinal=100, Tinterval=20, step=100000)
    lmp.write_input(output_dir=s)

For each system, we initialize a `Lammps` object

## Step 4: Creating `Job` objects and corresponding pbs files

In [ ]:
for s in smiles:
    job = nnmdkit.Job(jobname=s,
                      project='GT-rramprasad3-CODA20',
                      nodes=2,
                      ppn=24,
                      walltime='48:00:00',
                      LAMMPS_EXEC='~/p-rramprasad3-0/NNLMP/lmp')
    job.write_pbs(output_dir=s)